<a href="https://colab.research.google.com/github/Akshita0714/RMT-Correlation-Matrix-Cleanning/blob/main/Cleaning_Correlation_Matrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Defining the RMT methods (Github)

In [ ]:
%%capture --no-display
!pip install yfinance
!pip install yahoofinancials
!pip install rie_estimator

from __future__ import division, print_function
from builtins import reversed
from builtins import map, zip
from collections import MutableSequence, Sequence
import copy
import math
from math import ceil
from numbers import Complex, Integral, Real
import sys
import warnings
from sklearn.covariance import EmpiricalCovariance
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import LedoitWolf
import rie_estimator

import numpy as np
import matplotlib as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
import datetime as dt
import pandas_datareader

In [ ]:
def checkDesignMatrix(X):
    """
       Parameters
       ----------
       X: a matrix of shape (T, N), where T denotes the number
           of samples and N labels the number of features.
           If T < N, a warning is issued to the user, and the transpose
           of X is considered instead.
       Returns:
       T: type int
       N: type int
       transpose_flag: type bool
           Specify if the design matrix X should be transposed
           in view of having less rows than columns.       
    """
    
    try:
        assert isinstance(X, (np.ndarray, pd.DataFrame, pd.Series,
                              MutableSequence, Sequence))
    except AssertionError:
        raise
        sys.exit(1)

    #to convert input to an array. Input can be lists, lists of tuples, tuples, tuples of tuples, tuples of lists and arrays.
    X = np.asarray(X, dtype=float)
    #Scalar and 1-dimensional inputs are converted to 2-dimensional arrays, whilst higher-dimensional inputs are preserved.
    X = np.atleast_2d(X) 

    if X.shape[0] < X.shape[1]:
        warnings.warn("The Marcenko-Pastur distribution pertains to "
                      "the empirical covariance matrix of a random matrix X "
                      "of shape (T, N). It is assumed that the number of "
                      "samples T is assumed higher than the number of "
                      "features N. The transpose of the matrix X submitted "
                      "at input will be considered in the cleaning schemes "
                      "for the corresponding correlation matrix.", UserWarning)
        
        T, N = reversed(X.shape)
        transpose_flag = True
    else:
        T, N = X.shape
        transpose_flag = False
        
    return T, N, transpose_flag
    
def stieltjes(z, E):
    """
       Parameters
       ----------
       z: complex number
       E: square matrix
       Returns
       -------
       A complex number, the resolvent of square matrix E, 
       also known as its Stieltjes transform.
       Reference
       ---------
       "Financial Applications of Random Matrix Theory: a short review",
       J.-P. Bouchaud and M. Potters
       arXiv: 0910.1205 [q-fin.ST]
    """

    try:
        assert isinstance(z, Complex)
        
        assert isinstance(E, (np.ndarray, pd.DataFrame,
                              MutableSequence, Sequence))
        E = np.asarray(E, dtype=float)
        E = np.atleast_2d(E)
        assert E.shape[0] == E.shape[1]
    except AssertionError:
        raise
        sys.exit(1)

    N = E.shape[0]
    
    ret = z * np.eye(N, dtype=float) - E
    ret = np.trace(ret) / N

    return ret


# Cleaning Using Clipped Function
def marcenkoPastur(X):
    """
       Parameter
       ---------
       X: random matrix of shape (T, N), with T denoting the number
           of samples, whereas N refers to the number of features.
           It is assumed that the variance of the elements of X
           has been normalized to unity.           
       Returns
       -------
       (lambda_min, lambda_max): type tuple
           Bounds to the support of the Marcenko-Pastur distribution
           associated to random matrix X.
       rho: type function
           The Marcenko-Pastur density.
       Reference
       ---------
       "DISTRIBUTION OF EIGENVALUES FOR SOME SETS OF RANDOM MATRICES",
       V. A. Marcenko and L. A. Pastur
       Mathematics of the USSR-Sbornik, Vol. 1 (4), pp 457-483
    """

    T, N, _ = checkDesignMatrix(X)
    q = N / float(T)

    lambda_min = (1 - np.sqrt(q))**2
    lambda_max = (1 + np.sqrt(q))**2

    def rho(x):
        ret = np.sqrt((lambda_max - x) * (x - lambda_min))
        ret /= 2 * np.pi * q * x
        return ret if lambda_min < x < lambda_max else 0.0

    return (lambda_min, lambda_max), rho


def xiHelper(x, q, E):
    """Helper function to the rotationally-invariant, optimal shrinkage
       estimator of the true correlation matrix (implemented via function
       optimalShrinkage of the present module). 
       Parameters
       ----------
       x: type derived from numbers.Real
           Would typically be expected to be an eigenvalue from the
           spectrum of correlation matrix E. The present function
           can however handle an arbitrary floating-point number.
       q: type derived from numbers.Real
           The number parametrizing a Marcenko-Pastur spectrum.
       E: type numpy.ndarray
           Symmetric correlation matrix associated with the 
           Marcenko-Pastur parameter q specified above.
       Returns
       -------
       xi: type float
           Cleaned eigenvalue of the true correlation matrix C underlying
           the empirical correlation E (the latter being corrupted 
           with in-sample noise). This cleaned version is computed
           assuming no prior knowledge on the structure of the true
           eigenvectors (thereby leaving the eigenvectors of E unscathed). 
       References
       ----------
       * "Rotational invariant estimator for general noisy matrices",
         J. Bun, R. Allez, J.-P. Bouchaud and M. Potters
         arXiv: 1502.06736 [cond-mat.stat-mech]
       * "Cleaning large Correlation Matrices: tools from Random Matrix Theory",
         J. Bun, J.-P. Bouchaud and M. Potters
         arXiv: 1610.08104 [cond-mat.stat-mech]
    """

    try:
        assert isinstance(x, Real)
        assert isinstance(q, Real)
        assert isinstance(E, np.ndarray) and E.shape[0] == E.shape[1]
        assert np.allclose(E.transpose(1, 0), E)
    except AssertionError:
        raise
        sys.exit(1)

    N = E.shape[0]
    
    z = x - 1j / np.sqrt(N)
    s = stieltjes(z, E)
    xi = x / abs(1 - q + q * z * s)**2

    return xi

def clipped(X, alpha=None, return_covariance=False):
    """Clips the eigenvalues of an empirical correlation matrix E 
       in order to provide a cleaned estimator E_clipped of the 
       underlying correlation matrix.
       Proceeds by keeping the [N * alpha] top eigenvalues and shrinking
       the remaining ones by a trace-preserving constant 
       (i.e. Tr(E_clipped) = Tr(E)).
       Parameters
       ----------
       X: design matrix, of shape (T, N), where T denotes the number
           of samples (think measurements in a time series), while N
           stands for the number of features (think of stock tickers).
       alpha: type float or derived from numbers.Real (default: None)
           Parameter between 0 and 1, inclusive, determining the fraction
           to keep of the top eigenvalues of an empirical correlation matrix.
           If left unspecified, alpha is chosen so as to keep all the
           empirical eigenvalues greater than the upper limit of 
           the support to the Marcenko-Pastur spectrum. Indeed, such 
           eigenvalues can be considered as associated with some signal,
           whereas the ones falling inside the Marcenko-Pastur range
           should be considered as corrupted with noise and indistinguishable
           from the spectrum of the correlation of a random matrix.
           This ignores finite-size effects that make it possible
           for the eigenvalues to exceed the upper and lower edges
           defined by the Marcenko-Pastur spectrum (cf. a set of results
           revolving around the Tracy-Widom distribution)
           
       return_covariance: type bool (default: False)
           If set to True, compute the standard deviations of each individual
           feature across observations, clean the underlying matrix
           of pairwise correlations, then re-apply the standard
           deviations and return a cleaned variance-covariance matrix.
       Returns
       -------
       E_clipped: type numpy.ndarray, shape (N, N)
           Cleaned estimator of the true correlation matrix C underlying
           a noisy, in-sample estimate E (empirical correlation matrix
           estimated from X). This cleaned estimator proceeds through
           a simple eigenvalue clipping procedure (cf. reference below).
           
           If return_covariance=True, E_clipped corresponds to a cleaned 
           variance-covariance matrix.
       Reference
       ---------
       "Financial Applications of Random Matrix Theory: a short review",
       J.-P. Bouchaud and M. Potters
       arXiv: 0910.1205 [q-fin.ST]
    """

    try:
        if alpha is not None:
            assert isinstance(alpha, Real) and 0 <= alpha <= 1
            
        assert isinstance(return_covariance, bool)
    except AssertionError:
        raise
        sys.exit(1)
    
    T, N, transpose_flag = checkDesignMatrix(X)
    if transpose_flag:
        X = X.T
        
    if not return_covariance:
        X = StandardScaler(with_mean=False,
                           with_std=True).fit_transform(X)

    ec = EmpiricalCovariance(store_precision=False,
                             assume_centered=True)
    ec.fit(X)
    E = ec.covariance_
    
    if return_covariance:
        inverse_std = 1./np.sqrt(np.diag(E))
        E *= inverse_std
        E *= inverse_std.reshape(-1, 1)

    eigvals, eigvecs = np.linalg.eigh(E)
    eigvecs = eigvecs.T

    if alpha is None:
        (lambda_min, lambda_max), _ = marcenkoPastur(X)
        xi_clipped = np.where(eigvals >= lambda_max, eigvals, np.nan)
    else:
        xi_clipped = np.full(N, np.nan)
        threshold = int(ceil(alpha * N))
        if threshold > 0:
            xi_clipped[-threshold:] = eigvals[-threshold:]

    gamma = float(E.trace() - np.nansum(xi_clipped))
    gamma /= np.isnan(xi_clipped).sum()
    xi_clipped = np.where(np.isnan(xi_clipped), gamma, xi_clipped)

    E_clipped = np.zeros((N, N), dtype=float)
    for xi, eigvec in zip(xi_clipped, eigvecs):
        eigvec = eigvec.reshape(-1, 1)
        E_clipped += xi * eigvec.dot(eigvec.T)
        
    tmp = 1./np.sqrt(np.diag(E_clipped))
    E_clipped *= tmp
    E_clipped *= tmp.reshape(-1, 1)
    
    if return_covariance:
      std = 1./inverse_std
      E_clipped *= std
      E_clipped *= std.reshape(-1, 1)

    return E_clipped

def linear_shrinkage(X, alpha=None, return_covariance=False):
          try:
              if alpha is not None:
                  assert isinstance(alpha, Real) and 0 <= alpha <= 1
                  
              assert isinstance(return_covariance, bool)
          except AssertionError:
              raise
              sys.exit(1)
          
          T, N, transpose_flag = checkDesignMatrix(X)
          if transpose_flag:
              X = X.T
              
          if not return_covariance:
              X = StandardScaler(with_mean=False,
                                with_std=True).fit_transform(X)

          ec = LedoitWolf(assume_centered=False).fit(X)
          ec.fit(X)
          E = ec.covariance_
          return E

#linear_shrinkage(X,alpha=None, return_covariance=False)

# Part 1: Replicating Results: Dummy Try-out data
---
</br>

In [ ]:
# extracting the S&P500 data

#df = pandas_datareader.yahoo.daily.YahooDailyReader(symbols = ("TSLA", 'morningstar'), start='2000-01-01', end='2022-12-31')
#snp500 = yf.download('SPY', start='2000-01-01', end='2022-12-31')

In [ ]:
ticker_list=['DOW','AMZN','MSFT','TSLA','AAPL','IBM', 'GLD', 'XOM', 'TLT', 'SHY']

def getData(ticker):
    print (ticker)
    data = yf.download(ticker, start='2015-01-01', end='2022-12-31')
    data['ticker']=ticker
    return data

snp_list=[]
for ticker in ticker_list:
    snp_list.append(getData(ticker))

snp500=pd.concat(snp_list)
snp500.pop('Open')
snp500.pop('High')
snp500.pop('Low')
snp500.pop('Adj Close')
snp500.pop('Volume')
snp500.tail()

# calculating the daily returns
snp500['Daily Returns'] = snp500['Close'].pct_change()*100
snp500['SMA'] = snp500['Close'].rolling(window=5).mean()
snp500 = snp500.dropna()
snp500.head()

DOW
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
IBM
[*********************100%***********************]  1 of 1 completed
GLD
[*********************100%***********************]  1 of 1 completed
XOM
[*********************100%***********************]  1 of 1 completed
TLT
[*********************100%***********************]  1 of 1 completed
SHY
[*********************100%***********************]  1 of 1 completed


,Close,ticker,Daily Returns,SMA
Date,,,,
2019-03-26,48.849998,DOW,-0.610383,49.075999
2019-03-27,50.099998,DOW,2.558854,49.135999
2019-03-28,50.849998,DOW,1.497006,49.509999
2019-03-29,51.630001,DOW,1.533928,50.116000
2019-04-01,53.500000,DOW,3.621923,50.985999


In [ ]:
snp500_pivot=pd.pivot_table(snp500, values = 'SMA', index=['Date'], columns = 'ticker').reset_index()
snp500_pivot=snp500_pivot.dropna()
snp500_pivot=snp500_pivot.reset_index()
snp500_pivot.head()

ticker,index,Date,AAPL,AMZN,DOW,GLD,IBM,MSFT,SHY,TLT,TSLA,XOM
0,1063,2019-03-26,47.4915,89.393201,49.075999,124.210001,133.822180,118.072002,84.008000,124.103999,17.871200,80.894000
1,1064,2019-03-27,47.5070,89.077501,49.135999,124.104001,133.753348,117.922002,84.066000,124.792000,17.887600,80.698000
2,1065,2019-03-28,47.1885,88.619101,49.509999,123.748001,133.462717,117.264001,84.122000,125.514000,17.948933,80.487999
3,1066,2019-03-29,47.1335,88.778900,50.116000,123.356001,133.778204,117.442001,84.140001,125.830000,18.153334,80.551999
4,1067,2019-04-01,47.2585,89.178201,50.985999,122.678001,134.565967,117.714000,84.090001,125.692001,18.536800,80.914000


In [ ]:
'''
from sklearn.model_selection import train_test_split

train, test = train_test_split(snp500, test_size=0.2)

def split_train_test(dataset, validation_size):
    valid = dataset.iloc[-validation_size:, :]
    train_test = dataset.iloc[:-validation_size)]

    train_length = int(0.63 * len(train_test))

    # split into input and outputs
    train_X, train_y = train_test.iloc[:train_length, :-1], train_test.iloc[:train_length, -1]
    test_X, test_y = train_test.iloc[train_length:, :-1], train_test.iloc[train_length:, -1]
    valid_X, valid_y = valid.iloc[:, :-1], valid.iloc[:, -1]

    return train_test, valid, train_X, train_y, test_X, test_y, valid_X, valid_y
'''

In [ ]:
def RMT_methods(df):
  df1=df.copy()
  X=np.array(df1)
  RMT_clipped=clipped(X, alpha=None, return_covariance=True)
  RMT_original = np.cov(X.T @ X)

  #X_1 = StandardScaler(with_mean=True, with_std=True).fit_transform(X)
  cov = LedoitWolf().fit(X)
  RMT_linear_shrikage=cov.covariance_
  RMT_identity = np.identity(len(df1.columns))
  RMT_rie = rie_estimator.get_rie(df1, normalize = False)
  return [RMT_rie, RMT_clipped, RMT_linear_shrikage,RMT_original ,RMT_identity]

In [ ]:
def data_preprocess(df):
  df1=df.copy()
  df1=df1.drop(['Date','index','Year'], axis=1)
  df1[df1.columns]= df1[df1.columns].apply(np.log)
  df1=df1.diff()
  df1=df1.dropna()
  scaler = StandardScaler()
  df1[df1.columns] = StandardScaler().fit_transform(df1[df1.columns])
  return df1

def portfolio_result(df,n_assets):
      df['Date'] = pd.DatetimeIndex(df['Date'])
      df = df.sort_values(by = 'Date')
      df['Year'] = df['Date'].dt.year
      df_test=df[df['Year']==df['Year'].max()]
      df_train=df[df['Year']!=df['Year'].max()]
      for i in range(0,len(df_test),126):
          df_test_temp = df_test.iloc[0:i]
          df_test_final = df_test.iloc[i:]
          df_train_final=pd.concat([df_train,df_test_temp])
          df_train_final = data_preprocess(df_train_final)
          df_test_final = data_preprocess(df_test_final)
          cov_list=RMT_methods(df_train_final)
          df_return = df_train_final.pct_change(periods=63)
          df_return = df_return.values[-1:]
          portfolio_return = (np.array(df_return).T)/math.sqrt(n_assets)

          for j in range(0,len(cov_list)):
            weights = markowitz_weight(portfolio_return,cov_list[j])
            weights = weights/sum(weights)
            port_ret = outofsample_returns(weights,df_test_final)
            port_risk= outofsample_risk(weights,df_test_final)
            sharpe_ratio = port_ret/np.sqrt(port_risk)
            annual_return = fullperiod_returns(weights,df_test_final)
            print("The daily average return:",port_ret)
            print("The daily average risk:",port_risk)
            print("The Sharpe Ratio:",sharpe_ratio)
            print("The Annual_return:",annual_return)
      return


def outofsample_risk(weights,df):
  ans = 0 
  for i in range(0,len(df)):
      temp = np.array(df.iloc[i:i+1])
      ans += np.square(np.dot(temp,weights)[0][0])
  return ans/len(df)

def outofsample_returns(weights,df):
  ans = 0 
  for i in range(0,len(df)):
      temp = np.array(df.iloc[i:i+1])
      ans += np.dot(temp,weights)[0][0]
  return ans/len(df)

def fullperiod_returns(weights,df):
  ans = 0 
  df1=df.sum(axis = 0, skipna = True)
  temp=np.array(pd.DataFrame(df1).T)
  ans = np.dot(temp,weights)[0][0]
  return ans

def markowitz_weight(portfolio_return,cov_mat):
      num = np.dot(np.linalg.inv(cov_mat),portfolio_return)
      dem = np.dot(portfolio_return.T,num)
      return num/dem[0][0]

In [ ]:
portfolio_result(snp500_pivot,len(ticker_list))

The daily average return: 1.9539925233402754e-17
The daily average risk: 1.1416238898897546
The Sharpe Ratio: 1.8287796207016698e-17
The Annual_return: 1.3874054649042759e-14
The daily average return: -8.43769498715119e-18
The daily average risk: 1.106674045070169
The Sharpe Ratio: -8.020731159836315e-18
The Annual_return: 1.3935416661132364e-14
The daily average return: 6.039613253960852e-17
The daily average risk: 1.0983473217057071
The Sharpe Ratio: 5.762876091292452e-17
The Annual_return: 1.4044538418282214e-14
The daily average return: 2.864375403532904e-17
The daily average risk: 0.36641346980049105
The Sharpe Ratio: 4.7319943840467883e-17
The Annual_return: 6.547063760238985e-15
The daily average return: 7.771561172376097e-18
The daily average risk: 0.5118860289590722
The Sharpe Ratio: 1.086229595959641e-17
The Annual_return: 7.530164375439244e-15
The daily average return: -8.59527502935605e-17
The daily average risk: 541.6646035808054
The Sharpe Ratio: -3.6931296101434816e-18
T

# Replicating Results: Importing 500 stocks

In [ ]:
!pip install -q kaggle

In [ ]:
#create kaggle directory
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
#copy json file to kaggle folder
! cp kaggle.json ~/.kaggle

In [ ]:
#permiissonn for json to act
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rohanrao/nifty50-stock-market-data

 27% 5.00M/18.4M [00:00<00:00, 46.8MB/s]
100% 18.4M/18.4M [00:00<00:00, 120MB/s] 


In [ ]:
! unzip /content/nifty50-stock-market-data.zip

Archive:  /content/nifty50-stock-market-data.zip
  inflating: ADANIPORTS.csv          
  inflating: ASIANPAINT.csv          
  inflating: AXISBANK.csv            
  inflating: BAJAJ-AUTO.csv          
  inflating: BAJAJFINSV.csv          
  inflating: BAJFINANCE.csv          
  inflating: BHARTIARTL.csv          
  inflating: BPCL.csv                
  inflating: BRITANNIA.csv           
  inflating: CIPLA.csv               
  inflating: COALINDIA.csv           
  inflating: DRREDDY.csv             
  inflating: EICHERMOT.csv           
  inflating: GAIL.csv                
  inflating: GRASIM.csv              
  inflating: HCLTECH.csv             
  inflating: HDFC.csv                
  inflating: HDFCBANK.csv            
  inflating: HEROMOTOCO.csv          
  inflating: HINDALCO.csv            
  inflating: HINDUNILVR.csv          
  inflating: ICICIBANK.csv           
  inflating: INDUSINDBK.csv          
  inflating: INFRATEL.csv            
  inflating: INFY.csv                
 

In [ ]:
'''import os
import csv

dirpath = '/content'
output = 'output_file.csv'
with open(output, 'w') as outfile:
    csvout = csv.writer(outfile)
    csvout.writerow(['FileName', 'Content'])

    files = os.listdir(dirpath)

    for filename in files:
        with open(dirpath + '/' + filename) as afile:
            csvout.writerow([filename, afile.read()])
            afile.close()

    outfile.close()'''

In [ ]:
import pandas as pd
import glob
import os
path = r'/content' # use your path
all_files = glob.glob(os.path.join(path , "/*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

#frame = pd.concat(li, axis=0, ignore_index=True)

print(df.tail())

            Date      Symbol Series  Prev Close   Open    High     Low   Last  \
3317  2021-04-26  ADANIPORTS     EQ      725.35  733.0  739.65  728.90  729.2   
3318  2021-04-27  ADANIPORTS     EQ      730.75  735.0  757.50  727.35  748.6   
3319  2021-04-28  ADANIPORTS     EQ      749.15  755.0  760.00  741.10  743.4   
3320  2021-04-29  ADANIPORTS     EQ      746.25  753.2  765.85  743.40  746.4   
3321  2021-04-30  ADANIPORTS     EQ      746.75  739.0  759.45  724.50  726.4   

       Close    VWAP    Volume      Turnover    Trades  Deliverable Volume  \
3317  730.75  733.25   9390549  6.885658e+14  116457.0              838079   
3318  749.15  747.67  20573107  1.538191e+15  236896.0             1779639   
3319  746.25  751.02  11156977  8.379106e+14  130847.0             1342353   
3320  746.75  753.06  13851910  1.043139e+15  153293.0             1304895   
3321  730.05  743.35  12600934  9.366911e+14  132141.0             3514692   

      %Deliverble  
3317       0.0892  
3318

In [ ]:
#dropped columns one by one and overwrote to column name
df=df.drop(['%Deliverble'],axis=1)
print(df.head())

         Date      Symbol  Prev Close   Close
0  2007-11-27  MUNDRAPORT      440.00  962.90
1  2007-11-28  MUNDRAPORT      962.90  893.90
2  2007-11-29  MUNDRAPORT      893.90  884.20
3  2007-11-30  MUNDRAPORT      884.20  921.55
4  2007-12-03  MUNDRAPORT      921.55  969.30


In [ ]:
# calculating ita(RIE) and coparing it with the eigen values of the cleaned covariance matrix





# dividing total length of time series T_tot into n consecutive non-overlapping samples of length T_out



# def R_2(t, w):
#     """
#     out of sample variance of returns
#     """
#     temp_1 = 0
#     temp_2 = 0
#     for j in range(T_out):

#         for i in range(N):

#             temp_1 += (w_i[i] * X_i_t[i][j])

#         temp_2 += (temp_1**2)
#     return temp_2/T_out

# temp_3 = 0

# for j in range(N):
#     temp_3 += R_2(t_j, u_i)
#     ita_ora += temp_3


In [ ]:
def markowitz_weight(df,n_assets):
      df = df.sort_values(by = 'Date')
      df['Year'] = df['Date'].year
      df_test=df[df['Year']==df['Year'].max()]
      df_train=df[df['Year']!=df['Year'].max()]
      for i in range(0,df_test.size(),126):
          df_test_temp = df.iloc[0:i]
          df_test_final = df_train_final.iloc[i:]
          df_train_final=df_train,df_test_temp
          df_train_final = data_preprocess(df_train_final)
          df_test_final = data_preprocess(df_test_final)
          cov_list=covariance_calc(df_train_final)
          df2 = df_train_final.drop(['Date','index'], axis=1).pct_change(periods=63)
          df2=df2.values[-1:]
          portfolio_return=(np.array(df2).T)/math.sqrt(n_assets)


          weights = markowitz_weight(portfolio_return,cov_list[i])
          weights = weights/sum(weights)
          port_ret=outofsample_returns(weights,df_test_final,)
          port_risk=outofsample_risk(weights,df_test_final,5)
          sharpe_ratio = port_ret/np.sqrt(port_risk)
          for i in range


def outofsample_risk(weights,df):
  df[df.columns]= df[df.columns].apply(np.log)
  df=df.diff()
  df=df.dropna()
  ans = 0 
  for i in range(0,len(df)):
      temp = np.array(df.iloc[i:i+1])
      ans += np.square(np.dot(weights,temp)[0])
  return ans/len(df)

def outofsample_returns(weights,df):
  df[df.columns]= df[df.columns].apply(np.log)
  df=df.diff()
  df=df.dropna()
  ans = 0 
  for i in range(0,len(df)):
      temp = np.array(df.iloc[i:i+1])
      ans += np.dot(weights,temp)[0]
  return ans/len(df)



def markowitz_weight(portfolio_return,cov_mat):
      num = np.dot(np.linalg.inv(cov_mat),portfolio_return)
      dem = np.dot(portfolio_return.T,num)
      return num/dem

In [ ]:
def sharpe_ratio(rets, rfr, N):
    """
    Computes the annualized sharpe ratio for a set of returns
    
    Arguments:
        rets - return data
        rfr  - annualized risk-free rate
        N    - No. of periods per year (for example: N = 12 for a monthly return data)
    """
    # risk-free rate(per period) = [(1 + risk-free rate(annual))^(1 / no. of periods)] - 1
    rf_per_period= (1+ rfr)**(1/N)-1
    
    excess_return = rets - rf_per_period
    
    # calculating the annualized excess return using the method defined above
    ann_excess_return = annualize_rets(excess_return, N)
    
    # calculating the annualized volatility using the method defined above
    ann_vol = annualize_vol(excess_return, N)
    
    return ann_excess_return/ann_vol

#-----------------------------------

def annualize_rets(rets, N):
    """
    Annualizes a set of returns (rets)
    
    Arguments:
        rets - return data (DataFrame)
        N    - No. of periods per year (for example: N = 12 for a monthly return data)
    """
    compounded_growth = (1+rets).prod()
    # compounded growth = (1 + ret_1), (1 + ret_2), ....... , (1 + ret_n)
    n = rets.shape[0]
    
    # annualized return = (compounded growth)**(1/n)   ----->   if the returns entered are annual returns,     i.e. No. of periods = 1
    # annualized return = (compounded growth)**(N/n)   ----->   if the returns entered have          --->           No. of periods per year = N
    return compounded_growth**(N/n)-1

#-----------------------------------

def annualize_vol(rets, N):
    """
    Annualizes the volatility (vol) for a set of returns
    
    Arguments:
        rets - return data
        N    - No. of periods per year (for example: N = 12 for a monthly return data)
    """
    return rets.std() * (N ** 0.5)

#-----------------------------------

def portfolio_return(weights, rets):
    """
    Computes the return of a portfolio based on the returns and the weights of the constituents
    
    Arguments:
        weights - weight vector (a numpy array or Nx1 matrix)
        rets    - return data   (a numpy array or Nx1 matrix)
    """
    
    # weights.T          -->    weight vector transpose
    # weights.T @ rets   -->    Dot product of the weight vector transpose and the return vector
    return weights.T @ rets

#--------------------

def portfolio_vol(weights, covariance_matrix):
    """
    Computes the volatility of a portfolio from a covariance matrix and constituents' weights
    
    Arguments:
        weights           - weight vector     (a numpy array or Nx1 matrix)
        covariance_matrix - covariance matrix (an N x N matrix)
    """
    
    # weights.T          -->    weight vector transpose
    #
    # 2-security portfolio variance = (w_1 * sigma_1)^2 + (w_2 * sigma)^2 + 2(w_1 * w_2 * cov(1,2))   ---->   where, sigma = variance ^ 0.5
    #
    # N-security portfolio variance = (weights.T @ covariance_matrix @ weights)
    return (weights.T @ covariance_matrix @ weights)**0.5

In [ ]:
def train_test_split(df):
    df['year'] = df.index.year
    df = df.reset_index()
    df=df.sort_values('Date')
    year_list=list(df['year'].unique())
    year_list.sort()
    last_year = year_list[-1]
    train = df[df["year"] < last_year]
    test = df[df["year"] >= last_year]
    return train,test

train, test = train_test_split(snp500)
train['year'].unique()